 ChIP-seq analysis

# align with bowtie2

In [14]:
# more details in this presentation https://qcb.ucla.edu/collaboratory/workshops/w7-chip-seq-analysis/

In [ ]:
# This workshop is for biologists to get familiar with codeing. 
# By no mean this will conver the most advanced development in the field, but I will share my experience, learning bioinformatics as a wet lab biologist. 
# And will mention some new trends in the field for future direction to look into. 

# I am also not native English speaker, be aware of typo issues and let me know. 
# I work at NIAID. this training is more of personal training. 
# Let me know if you have suggestions. 

# runing time of some of the computation can be long, we will not wait until if finishes. 
# I have intermediate cells, you can copy and from the shared folder. 
# There will be a month of time for you to try, after the workshop. 
# you can contact me at zhuy16@nih.gov.

In [2]:
pwd

/spin1/users/classes/NIEHS_NGS/RNA-workshop-2021/chip-seq


In [16]:
#ls

In [4]:
module load sratoolkit/2.9.6

[+] Loading sratoolkit 2.9.6  ... 


In [6]:
# fastq-dump -Z SRR502329 > data/STAT1_30m_IFNa.fastq

In [7]:
# fastq-dump -Z SRR502327 > data/STAT1_6h_IFNa.fastq
# fastq-dump -Z SRR502228 > data/INP_30m_IFNa.fastq
# fastq-dump -Z SRR502225 > data/INP_6h_IFNa.fastq

In [9]:
#rm STAT1_30m_IFNa.fastq

In [29]:
module load bowtie/2 samtools

[-] Unloading bowtie  2-2.4.4 
[+] Loading bowtie  2-2.4.4 
[+] Loading samtools 1.12  ... 


In [30]:
ls /fdb/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/

genome.1.bt2  genome.3.bt2  genome.fa      genome.rev.1.bt2
genome.2.bt2  genome.4.bt2  genome.fa.fai  genome.rev.2.bt2


In [31]:
export BOWTIE2_INDEXES=/fdb/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/

In [ ]:
# the following step takes a long time to finish 

In [33]:
# bowtie2 --phred64 -x hg19/hg19 --threads=$SLURM_CPUS_PER_TASK --no-unal --end-to-end --sensitive  -U $BOWTIE_TEST_DATA/ENCFF001KPB.fastq.gz| samtools view -q30 -Sb - > ENCFF001KPB.bam
# /fdb/igenomes/Homo_sapiens/UCSC/hg19/Sequence/Bowtie2Index/
# bowtie2 –q -p 4 –k 1 –-local –-no-unal –x hg19/hg19 STAT1_30m_IFNa.fastq -S  -S STAT1_30m_IFNa.sam

# optimized for biowulf

bowtie2  -x genome --threads=4  --no-unal --local -U fastq/STAT1_30m_IFNa.fastq -S 0r_STAT1_30m_IFNa.sam

# batch processing

In [37]:
#mkdir 0r_script_align

: 1

In [43]:
#cd ..

In [44]:
#rm -rf 0r_script_align/*

In [45]:
cd fastq

In [46]:
#cat gen_script_align.sh:

#!/bin/sh
for i in *.fastq 
do
cat <<EOT >> ../0r_script_align/script_align_$(echo $i | sed 's/.fastq//g').sh
#!/bin/sh
module load samtools
module load bowtie/2
bowtie2  -x genome --threads=4  --no-unal --local -U fastq/$i -S sam/$(echo $i | sed 's/.fastq//g').sam
EOT
done

In [50]:
cd ..

In [51]:
for i in 0r_script_align/script_align_*.sh; do echo $i; done

0r_script_align/script_align_INP_30m_IFNa.sh
0r_script_align/script_align_INP_6h_IFNa.sh
0r_script_align/script_align_STAT1_30m_IFNa.sh
0r_script_align/script_align_STAT1_6h_IFNa.sh


In [52]:
for i in 0r_script_align/script_align_*.sh; do sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 $i; done

17070404
17070405
17070409
17070411


In [53]:
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_STAT1_30m_IFNa.sh 
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_STAT1_6h_IFNa.sh 
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_INP_30m_IFNa.sh 
# sbatch --cpus-per-task=4 --mem=10g --gres=lscratch:10 0r_script_align/script_align_INP_6h_IFNa.sh 

# remove mitochondrial reads, unknown chr reads

In [5]:
module load samtools

[+] Loading samtools 1.12  ... 


In [1]:
cat bam.sh

sed '/chrM/d;/chrUn/d' $1.sam > $1_filtered.sam
samtools view -S -b $1_filtered.sam > $1.bam
samtools view -c $1.bam


In [ ]:
./bam.sh sam/STAT1_30m_IFNa
#10848585
./bam.sh sam/STAT1_6h_IFNa
#11100766           
./bam.sh sam/INP_30m_IFNa
#19147013
./bam.sh sam/INP_6h_IFNa
#19769206

In [8]:
#samtools view -c done_sam/INP_6h_IFNa.bam

19769206


In [ ]:
# normalization to library size

In [ ]:
samtools view -b -s 1.538 sam/INP_30m_IFNa.bam > sam/INP_30m_IFNa_11E6.bam
samtools view -b -s 1.564 sam/INP_6h_IFNa.bam > sam/INP_6h_IFNa_11E6.bam